In [47]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
import pandas as pd
import numpy as np

In [32]:
train_df = pd.read_csv('data/cleaned_train.csv')

# Extract the columns you want to use as input features
columns = ['BERT_sentiment_score', 'normalised_word_count', 'pub_day', 'pub_hour']
# Extract input features from the dataframe
x_train = train_df[columns].dropna()

y_train = np.array(train_df['n_comments'])

In [41]:
from sklearn.preprocessing import LabelEncoder
# Concatenate train and test data vertically to ensure consistent label encoding
combined_df = pd.concat([train_df['topic'], test_df['topic']], axis=0)

# Create an instance of LabelEncoder
le = LabelEncoder()

# Fit and transform the combined data using LabelEncoder
combined_encoded = le.fit_transform(combined_df)

# Apply the LabelEncoder transformation separately on train and test data
train_topic_encoded = np.array(le.transform(train_df['topic'].dropna()))
test_topic_encoded = le.transform(test_df['topic'].dropna())

In [49]:
train_topic_encoded
arr_df = pd.DataFrame(train_topic_encoded, columns=['topic encoded'])

In [8]:
#one-hot encoded string to tensor
data = train_df['topic_encoded'].dropna()

# Function to remove square brackets and convert string to list of integers
def preprocess_data(data):
    data = data.strip('[]')
    data_list = list(map(int, data))
    return data_list

# Apply preprocessing to each element in the column
data_list = data.apply(preprocess_data)

# Convert list of lists to numpy array
data_np = np.array(data_list.tolist())

# Convert numpy array to tensor
data_tensor = tf.convert_to_tensor(data_np, dtype=tf.float32)

# Reshape tensor to desired shape (e.g., (num_samples, num_features) in this case)
data_tensor = tf.reshape(data_tensor, (data_np.shape[0], -1))

In [3]:
import pandas as pd
from gensim.models import Word2Vec

# Load your text data
data = train_df['topic'].dropna()

# Convert the text data into list of sentences, where each sentence is a list of words
sentences = [sent.split() for sent in data]

# Train a Word2Vec model on your text data
embedding_dim = 100  # Specify the embedding dimension
model_train = Word2Vec(sentences, vector_size=embedding_dim, window=5, min_count=1, workers=4)

# Get the word embeddings for your topics
topic_embeddings = []
for topic in data:
    topic_words = topic.split()  # Split the topic text into individual words
    topic_vector = np.zeros(embedding_dim)  # Initialize a vector of zeros for the topic
    num_words = 0  # Keep track of the number of words in the topic

    # Iterate over each word in the topic and add its embedding to the topic vector
    for word in topic_words:
        if word in model_train.wv:
            topic_vector += model_train.wv[word]
            num_words += 1

    # Average the word embeddings to get the final topic embedding
    if num_words > 0:
        topic_vector /= num_words

    topic_embeddings.append(topic_vector)

# Convert the topic embeddings list into a numpy array
topic_embeddings = np.array(topic_embeddings)

# Now, topic_embeddings contains the numerical representations (embeddings) for your "topic" text data, which you can concatenate with your other input features and use as input to your MLP model for training and prediction.


In [4]:
topic_embeddings[0]

array([-8.61968752e-03,  3.66573804e-03,  5.18988352e-03,  5.74193709e-03,
        7.46691693e-03, -6.16767630e-03,  1.10561366e-03,  6.04728237e-03,
       -2.84005166e-03, -6.17352268e-03, -4.10223001e-04, -8.36895034e-03,
       -5.60001377e-03,  7.10453745e-03,  3.35253961e-03,  7.22566852e-03,
        6.80024642e-03,  7.53074186e-03, -3.78915551e-03, -5.61807130e-04,
        2.34837527e-03, -4.51903325e-03,  8.38873163e-03, -9.85816494e-03,
        6.76464103e-03,  2.91441684e-03, -4.93283290e-03,  4.39818623e-03,
       -1.73957588e-03,  6.71138288e-03,  9.96484980e-03, -4.36244486e-03,
       -5.99339022e-04, -5.69563871e-03,  3.85082234e-03,  2.78662681e-03,
        6.89107645e-03,  6.10109558e-03,  9.53849591e-03,  9.27341729e-03,
        7.89806806e-03, -6.98950514e-03, -9.15586483e-03, -3.55753902e-04,
       -3.09984200e-03,  7.89431576e-03,  5.93857281e-03, -1.54566288e-03,
        1.51096343e-03,  1.79003959e-03,  7.81757105e-03, -9.51018836e-03,
       -2.05531120e-04,  

In [50]:
# Concatenate the 'topic_encoded' tensor with the other input features
x_train = pd.concat([x_train, arr_df], axis=1)

In [51]:
x_train.shape

(12461, 5)

In [52]:
# Create MLP model
model = Sequential()
num_features = 5
model.add(Dense(64, activation='relu', input_dim=num_features))
model.add(Dense(32, activation='relu'))
model.add(Dense(1, activation='linear'))

In [53]:
# Compile the model
model.compile(optimizer='adam', loss='mean_squared_error')

In [54]:
# Train the model
model.fit(x_train, y_train, batch_size=32, epochs=100, validation_split=0.2)

Epoch 1/100
312/312 [==============================] - 1s 2ms/step - loss: 301654.7500 - val_loss: 241184.0625
Epoch 2/100
312/312 [==============================] - 1s 2ms/step - loss: 276778.1250 - val_loss: 239362.1094
Epoch 3/100
312/312 [==============================] - 1s 2ms/step - loss: 272539.9375 - val_loss: 237119.5156
Epoch 4/100
312/312 [==============================] - 1s 2ms/step - loss: 267703.7500 - val_loss: 236943.3750
Epoch 5/100
312/312 [==============================] - 1s 2ms/step - loss: 263810.6562 - val_loss: 232204.6250
Epoch 6/100
312/312 [==============================] - 1s 2ms/step - loss: 261074.4375 - val_loss: 230642.5781
Epoch 7/100
312/312 [==============================] - 1s 2ms/step - loss: 258726.0312 - val_loss: 229503.0469
Epoch 8/100
312/312 [==============================] - 1s 2ms/step - loss: 256253.7656 - val_loss: 227662.8125
Epoch 9/100
312/312 [==============================] - 1s 2ms/step - loss: 254279.8281 - val_loss: 225122.5312
E

312/312 [==============================] - 1s 2ms/step - loss: 234304.8438 - val_loss: 215354.7344
Epoch 75/100
312/312 [==============================] - 1s 2ms/step - loss: 234164.9844 - val_loss: 221960.5156
Epoch 76/100
312/312 [==============================] - 1s 2ms/step - loss: 234354.1562 - val_loss: 217995.9844
Epoch 77/100
312/312 [==============================] - 1s 2ms/step - loss: 234488.9062 - val_loss: 218325.8906
Epoch 78/100
312/312 [==============================] - 1s 3ms/step - loss: 233407.2344 - val_loss: 216950.6562
Epoch 79/100
312/312 [==============================] - 1s 3ms/step - loss: 233611.3281 - val_loss: 217479.1875
Epoch 80/100
312/312 [==============================] - 1s 2ms/step - loss: 233581.6094 - val_loss: 215795.7344
Epoch 81/100
312/312 [==============================] - 1s 2ms/step - loss: 234604.1719 - val_loss: 216016.5312
Epoch 82/100
312/312 [==============================] - 1s 2ms/step - loss: 233004.9219 - val_loss: 223107.4375
Epoch

In [13]:
# save the trained model
model.save('MLP_prediction_model.h5')

In [55]:
test_df = pd.read_csv('data/cleaned_test.csv')

x_test = test_df[columns].dropna().values

In [45]:
#one-hot encoded string to tensor
data = test_df['topic_encoded'].dropna()

# Function to remove square brackets and convert string to list of integers
def preprocess_data(data):
    data = data.strip('[]')
    data_list = list(map(int, data))
    return data_list

# Apply preprocessing to each element in the column
data_list = data.apply(preprocess_data)

# Convert list of lists to numpy array
data_np = np.array(data_list.tolist())

# Convert numpy array to tensor
data_tensor = tf.convert_to_tensor(data_np, dtype=tf.float32)

# Reshape tensor to desired shape (e.g., (num_samples, num_features) in this case)
data_tensor = tf.reshape(data_tensor, (data_np.shape[0], -1))

In [11]:
# Load your text data
data = test_df['topic'].dropna()

# Convert the text data into list of sentences, where each sentence is a list of words
sentences = [sent.split() for sent in data]

# Train a Word2Vec model on your text data
embedding_dim = 100  # Specify the embedding dimension
model_test = Word2Vec(sentences, vector_size=embedding_dim, window=5, min_count=1, workers=4)

# Get the word embeddings for your topics
topic_embeddings = []
for topic in data:
    topic_words = topic.split()  # Split the topic text into individual words
    topic_vector = np.zeros(embedding_dim)  # Initialize a vector of zeros for the topic
    num_words = 0  # Keep track of the number of words in the topic

    # Iterate over each word in the topic and add its embedding to the topic vector
    for word in topic_words:
        if word in model_test.wv:
            topic_vector += model_test.wv[word]
            num_words += 1

    # Average the word embeddings to get the final topic embedding
    if num_words > 0:
        topic_vector /= num_words

    topic_embeddings.append(topic_vector)

# Convert the topic embeddings list into a numpy array
topic_embeddings = np.array(topic_embeddings)

In [56]:
test_arr_df = pd.DataFrame(test_topic_encoded, columns=['topic encoded'])

In [57]:
# Concatenate the 'topic_encoded' tensor with the other input features
x_test = tf.concat([x_test, test_arr_df], axis=1)

In [58]:
y_pred = model.predict(x_test)

122/122 [==============================] - 0s 975us/step


In [63]:
y_pred[0]

array([834.61005], dtype=float32)